<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Pipeline-part" data-toc-modified-id="Pipeline-part-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pipeline part</a></span></li><li><span><a href="#Backend-part" data-toc-modified-id="Backend-part-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Backend part</a></span></li></ul></div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

# Pipeline part

In [ ]:
import pandas as pd
import glob

In [ ]:
!pwd

In [ ]:
article_snapshots = glob.glob("../articles/articles-*")

In [ ]:
len(article_snapshots)

In [ ]:
from pipeline.text_processing import load_article_snapshots

In [ ]:
df = load_article_snapshots(article_snapshots)

In [ ]:
from pipeline.text_processing import add_lemmatized_texts, fit_tf_idf

In [ ]:
add_lemmatized_texts(df)

In [ ]:
X, words = fit_tf_idf(df["lemmatized_texts"])

In [ ]:
words

# Backend part

In [ ]:
from backend.recommender.recommend import (
    recommend,
    get_daily_google_trends,
    get_relevant_words,
    estimate_popularity,
    calculate_frecency
)

In [ ]:
top_trends = get_daily_google_trends()

In [ ]:
top_trends

In [ ]:
pop = estimate_popularity(top_trends, X, words)

In [ ]:
(pd.Series(pop.squeeze()) == 0).value_counts()

In [ ]:
import time
import numpy as np
from math import log, e

In [ ]:
def calculate_frecency(popularity, age):
    """
    see https://wiki.mozilla.org/User:Jesse/NewFrecency
    """
    # how much will be older articles penalized,
    # interpretation: the denomintaor is number of seconds after which the score halves
    lambda_const = log(2) / (7*24*60*60)  # 7 days
    return np.multiply(np.log(popularity), np.exp(-lambda_const * age))


In [ ]:
daily_trends = get_daily_google_trends()
popularity = estimate_popularity(daily_trends, X, words) + 1
age = time.time() - df.published.map(time.mktime)

frecency = np.squeeze(np.asarray(calculate_frecency(popularity.T, age.values)))


In [ ]:
(pd.Series(frecency) == 0).value_counts()

In [ ]:
recommend(df, X, words)

In [ ]:
top_ids = frecency.argsort()[::-1][:10]
df.iloc[top_ids[:10], 0].values

In [ ]:
df.sort_values("published", ascending=False).title.values[:20]